In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = mls
band = [11]
commitnumber = de5d43e
molecule = {'o3': 'atmpro'}
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o o3 band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 2e-20
  molecule = o3
  ng_adju = [0, 0]
  ng_refs = [2, 5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.55, 1.55), (1.55, 1.55, 1.55, 1.55, 1.8)]
  wgt = [(0.3, 0.4), (0.5, 0.6, 0.7, 0.85, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`              
            layer    coolrg                layer        coolrg
pressure                                                      
0.000312      1.0 -0.000186                  NaN           NaN
0.000312      NaN       NaN                  1.0 -1.550224e-04
0.000750      2.0 -0.000208                  2.0 -1.780381e-04
0.001052      3.0 -0.000253                  3.0 -2.087905e-04
0.001476      4.0 -0.000297                  4.0 -2.450519e-04
0.002070      5.0 -0.000342                  5.0 -2.818741e-04
0.002904      6.0 -0.000386                  6.0 -3.189189e-04
0.004074      7.0 -0.000430                  7.0 -3.558223e-04
0.005714      8.0 -0.000475                  8.0 -3.928910e-04
0.008015      9.0 -0.000518                  9.0 -4.297443e-04
0.011243     10.0 -0.000562                 10.0 -4.667477e-04
0.015771     11.0 -0.000606                 11.0 -5.036239e-04
0.022122     12.0 -0.000649                 12.0 -5.405399e-04
0.031031     13.0 -0.000693                 13.0 -5.774279e-04
0.043528     14.0 -0.000735                 14.0 -6.148777e-04
0.061057     15.0 -0.000793                 15.0 -6.674832e-04
0.085645     16.0 -0.000932                 16.0 -7.963250e-04
0.120136     17.0 -0.001073                 17.0 -9.449102e-04
0.168516     18.0 -0.001064                 18.0 -9.909102e-04
0.236378     19.0 -0.000797                 19.0 -8.495246e-04
0.331549     20.0 -0.000103                 20.0 -3.900317e-04
0.465100     21.0  0.001272                 21.0  5.844079e-04
0.652400     22.0  0.003683                 22.0  2.354998e-03
0.915100     23.0  0.007063                 23.0  4.871268e-03
1.283650     24.0  0.009073                 24.0  6.316159e-03
1.800600     25.0  0.007894                 25.0  5.285420e-03
2.525700     26.0  0.005016                 26.0  2.887987e-03
3.542800     27.0  0.001139                 27.0 -3.492057e-04
4.969550     28.0 -0.002607                 28.0 -3.541884e-03
6.970850     29.0 -0.005579                 29.0 -6.234981e-03
9.778100     30.0 -0.007978                 30.0 -8.553709e-03
13.715850    31.0 -0.009738                 31.0 -1.032423e-02
19.239350    32.0 -0.010447                 32.0 -1.102402e-02
26.987250    33.0 -0.010374                 33.0 -1.090526e-02
37.855300    34.0 -0.009985                 34.0 -1.038087e-02
53.100050    35.0 -0.008688                 35.0 -8.810265e-03
73.887500    36.0 -0.006264                 36.0 -6.124058e-03
97.662500    37.0 -0.004080                 37.0 -3.852393e-03
121.437500   38.0 -0.002864                 38.0 -2.637945e-03
145.212500   39.0 -0.002249                 39.0 -2.031855e-03
168.987500   40.0 -0.001772                 40.0 -1.576728e-03
192.762500   41.0 -0.001338                 41.0 -1.178897e-03
216.537500   42.0 -0.001059                 42.0 -9.265062e-04
240.312500   43.0 -0.000884                 43.0 -7.691681e-04
264.087500   44.0 -0.000723                 44.0 -6.269877e-04
287.862500   45.0 -0.000599                 45.0 -5.178280e-04
311.637500   46.0 -0.000522                 46.0 -4.502126e-04
335.412500   47.0 -0.000457                 47.0 -3.934664e-04
359.187500   48.0 -0.000396                 48.0 -3.397246e-04
382.962500   49.0 -0.000343                 49.0 -2.941661e-04
406.737500   50.0 -0.000299                 50.0 -2.562726e-04
430.512500   51.0 -0.000258                 51.0 -2.209311e-04
454.287500   52.0 -0.000220                 52.0 -1.881335e-04
478.062500   53.0 -0.000185                 53.0 -1.579223e-04
501.837500   54.0 -0.000154                 54.0 -1.318774e-04
525.612500   55.0 -0.000127                 55.0 -1.090501e-04
549.387500   56.0 -0.000102                 56.0 -8.767852e-05
573.162500   57.0 -0.000079                 57.0 -6.801074e-05
596.937500   58.0 -0.000057                 58.0 -4.961373e-05
620.712500   59.0 -0.000037                 59.0 -3.211581e-05
644.487500   60.0 -0.000017                 6

# Fluxes by Layer

CRD                            CLIRAD            \
                        flug          fldg     fnetg      flug      fldg   
pressure     level                                                         
0.000000e+00 1     -6.480971  0.000000e+00 -6.480971       NaN       NaN   
1.000000e-08 1           NaN           NaN       NaN -6.484434  0.000002   
6.244000e-04 2     -6.480971  7.533486e-10 -6.480971 -6.484434  0.000002   
8.759000e-04 3     -6.480971  1.099333e-09 -6.480971 -6.484434  0.000002   
1.228600e-03 4     -6.480971  1.710463e-09 -6.480971 -6.484434  0.000002   
1.723400e-03 5     -6.480971  2.759287e-09 -6.480971 -6.484434  0.000002   
2.417400e-03 6     -6.480971  4.518321e-09 -6.480971 -6.484434  0.000002   
3.390900e-03 7     -6.480971  7.419061e-09 -6.480971 -6.484434  0.000002   
4.756500e-03 8     -6.480971  1.213817e-08 -6.480971 -6.484434  0.000002   
6.672000e-03 9     -6.480971  1.973169e-08 -6.480971 -6.484434  0.000002   
9.358900e-03 10    -6.480972  3.184167e-08 -6.480972 -6.484434  0.000002   
1.312780e-02 11    -6.480972  5.100957e-08 -6.480972 -6.484434  0.000002   
1.841450e-02 12    -6.480972  8.116103e-08 -6.480972 -6.484435  0.000002   
2.583020e-02 13    -6.480973  1.283286e-07 -6.480973 -6.484435  0.000002   
3.623230e-02 14    -6.480974  2.017745e-07 -6.480974 -6.484436  0.000002   
5.082340e-02 15    -6.480975  3.159208e-07 -6.480975 -6.484437  0.000002   
7.129060e-02 16    -6.480977  5.118434e-07 -6.480977 -6.484439  0.000003   
1.000000e-01 17    -6.480981  9.683479e-07 -6.480980 -6.484442  0.000003   
1.402710e-01 18    -6.480987  2.213090e-06 -6.480985 -6.484447  0.000004   
1.967600e-01 19    -6.480998  5.447183e-06 -6.480992 -6.484457  0.000006   
2.759970e-01 20    -6.481013  1.349474e-05 -6.481000 -6.484470  0.000012   
3.871000e-01 21    -6.481034  3.283527e-05 -6.481001 -6.484490  0.000027   
5.431000e-01 22    -6.481056  7.799015e-05 -6.480977 -6.484513  0.000060   
7.617000e-01 23    -6.481063  1.803415e-04 -6.480882 -6.484530  0.000138   
1.068500e+00 24    -6.481025  3.991440e-04 -6.480625 -6.484520  0.000306   
1.498800e+00 25    -6.480947  7.838489e-04 -6.480163 -6.484498  0.000605   
2.102400e+00 26    -6.480923  1.324373e-03 -6.479598 -6.484553  0.001039   
2.949000e+00 27    -6.481096  2.000861e-03 -6.479096 -6.484830  0.001605   
4.136600e+00 28    -6.481667  2.731503e-03 -6.478935 -6.485527  0.002253   
5.802500e+00 29    -6.482861  3.411002e-03 -6.479450 -6.486871  0.002898   
8.139200e+00 30    -6.484992  3.997888e-03 -6.480994 -6.489194  0.003495   
1.141700e+01 31    -6.488578  4.486571e-03 -6.484092 -6.493043  0.004023   
1.601470e+01 32    -6.494254  4.858928e-03 -6.489395 -6.499095  0.004450   
2.246400e+01 33    -6.502488  5.112052e-03 -6.497376 -6.507829  0.004762   
3.151050e+01 34    -6.513803  5.309882e-03 -6.508493 -6.519762  0.005007   
4.420010e+01 35    -6.529015  5.512211e-03 -6.523503 -6.535601  0.005240   
6.200000e+01 36    -6.547523  5.701606e-03 -6.541822 -6.554388  0.005450   
8.577500e+01 37    -6.565306  5.842980e-03 -6.559463 -6.571794  0.005606   
1.095500e+02 38    -6.576888  5.934955e-03 -6.570953 -6.582745  0.005706   
1.333250e+02 39    -6.585029  6.009336e-03 -6.579019 -6.590254  0.005785   
1.571000e+02 40    -6.591425  6.072955e-03 -6.585352 -6.596043  0.005851   
1.808750e+02 41    -6.596474  6.132156e-03 -6.590342 -6.600545  0.005912   
2.046500e+02 42    -6.600311  6.200307e-03 -6.594110 -6.603930  0.005976   
2.284250e+02 43    -6.603382  6.289240e-03 -6.597093 -6.606619  0.006056   
2.522000e+02 44    -6.605982  6.400808e-03 -6.599581 -6.608882  0.006153   
2.759750e+02 45    -6.608146  6.528539e-03 -6.601617 -6.610757  0.006262   
2.997500e+02 46    -6.609974  6.670224e-03 -6.603304 -6.612336  0.006382   
3.235250e+02 47    -6.611606  6.830876e-03 -6.604775 -6.613739  0.006517   
3.473000e+02 48    -6.613073  7.010288e-03 -6.606063 -6.614997  0.006667   
3.710750e+02 49    -6.614382  7.205535e-03 -6.607176 -6.616117  0.006830 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')